In [12]:
import numpy as np
import matplotlib.pylab as py
import matplotlib.pyplot as plt
from qiskit.visualization import *
from qiskit import *
from qiskit.aqua import *
from qiskit.quantum_info import *
from qiskit.quantum_info.states import *
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_textbook.tools import array_to_latex
from qiskit.extensions import RXGate, XGate, CXGate,IGate

In [15]:
#Prepare Singlet Projector
singlet_psi = Statevector.from_label('0+')-Statevector.from_label('+0')
singlet_proj= singlet_psi.to_operator()
#Prepare Triplet Projector
i=Operator(IGate())
i_op=i.expand(i)
triplet_proj=i_op-singlet_proj

In [497]:
def generate_permutations(values, occurrences):
    if sum(occurrences) == 0:
        yield []

    for i, (v, o) in enumerate(zip(values, occurrences)):
        if o > 0:
            occurrences[i] -= 1
            for p in generate_permutations(values, occurrences):
                yield [v] + p
            occurrences[i] += 1
def compute_triplet_profile(projectors, state):

    return [sum([expect(p, state) for p in ps]) for ps in projectors]

def generate_projectors(site_number):
    
    return [
        [
            tensor(perm)
            for perm in generate_permutations(
                [singlet_proj, triplet_proj], [site_number // 2 - i, i]
            )
        ]
        for i in range(site_number // 2 + 1)
    ]


In [496]:
#Define the Hamiltonian
N=2
J1=1
J2=0.5

q=QuantumRegister(N)
c=ClassicalRegister(N)
qc=QuantumCircuit(q,c)


for i in range(0,N-1,1):
    qc.cx(i,i+1)
    qc.u1(J1*np.pi,i+1)
    qc.cx(i,i+1)

qc.barrier()
for i in range(0,N-1,1):
    qc.h(i)
    qc.h(i+1)
    qc.cx(i,i+1)
    qc.u1(J1*np.pi,i+1)
    qc.cx(i,i+1)
    qc.h(i)
    qc.h(i+1)

#Nearest-neigbour interaction Z+X
"""for i in range(0,N,2):
    qc.cx(i,i+1)
    qc.u1(J1*np.pi,i+1)
    qc.u2(J1*np.pi,np.pi,i+1)
    qc.cx(i,i+1)
    
for i in range(1,N-1,2):
    qc.cx(i,i+1)
    qc.u1(J1*np.pi,i+1)
    qc.u2(J1*np.pi,np.pi,i+1)
    qc.cx(i,i+1)"""

backend = BasicAer.get_backend('unitary_simulator')
job = execute(qc, backend)
unitary=job.result().get_unitary(qc, decimals=3)
array_to_latex(unitary, pretext="\\text{Circuit = }\n")

#plt.plot(list(range(N // 2 + 1)),compute_triplet_profile(generate_projectors(N), gs[0]),label=f"J2 ={round(J2,2)}")
#plt.xlabel('$m_{t}$')
#plt.ylabel('$p(m_{t})$')
#plt.legend()

<IPython.core.display.Math object>